In [2]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/artem-gordeev/ColdAtoms.jl"))

    Updating git-repo `https://github.com/artem-gordeev/ColdAtoms.jl`
   Resolving package versions...
  No Changes to `C:\Users\А\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\А\.julia\environments\v1.11\Manifest.toml`


In [3]:
using ColdAtoms